# Risco de fogo no Distrito Federal por região no ano de 2019

In [1]:
import os

import pandas as pd
import numpy as np
import netCDF4 as nc

import datetime

### Define área de interesse e resolução do píxel:

In [2]:
lonW = -73.99 # Longitude Oeste.
lonE = -33.86 # Longitude Leste.
latS = -28.63 # Latitude Sul.
latN = 5.29   # Latitude Norte.
dgrad = .25   # Resolução do píxel.

In [3]:
df = pd.read_csv('DF-2019.csv', parse_dates=['datahora'])#, date_parser=dateparse)

In [4]:
df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2019-01-08 04:30:00,NPP-375,Brasil,DISTRITO FEDERAL,BRASILIA,Cerrado,3,0.0,0.95,-15.58003,-47.38740,1.3
1,2019-01-08 16:08:21,NOAA-20,Brasil,DISTRITO FEDERAL,BRASILIA,Cerrado,0,0.0,0.00,-15.59302,-47.39392,NaN
2,2019-01-08 16:08:21,NOAA-20,Brasil,DISTRITO FEDERAL,BRASILIA,Cerrado,0,0.0,0.00,-15.59359,-47.39852,NaN
3,2019-01-08 17:06:00,NPP-375,Brasil,DISTRITO FEDERAL,BRASILIA,Cerrado,3,0.0,0.86,-15.59054,-47.39420,17.0
4,2019-01-08 17:06:00,NPP-375,Brasil,DISTRITO FEDERAL,BRASILIA,Cerrado,3,0.0,0.86,-15.59129,-47.39827,17.0


In [5]:
df.describe()

,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
count,3751.000000,3751.000000,3600.000000,3751.000000,3751.000000,1926.000000
mean,24.295121,0.127942,3.839575,-15.770668,-47.923721,20.973053
std,28.627466,1.268868,50.066111,0.159648,0.208020,53.208105
min,0.000000,0.000000,0.000000,-16.050220,-48.275425,0.300000
25%,0.000000,0.000000,0.000000,-15.921490,-48.120000,3.500000
50%,8.000000,0.000000,1.000000,-15.763700,-47.912500,8.400000
75%,47.000000,0.000000,1.000000,-15.631355,-47.790685,20.400000
max,120.000000,27.690000,777.700000,-15.500760,-47.358570,1223.300000


### As colunas alvos são as de: latitude, longitude e tempo.

In [6]:
lat_col = 'latitude'
lon_col = 'longitude'
time_col = 'datahora'

### Cria um filtro com a área de interesse:

In [7]:
filter_ = (df[lat_col] < latN) & (df[lat_col] > latS) & \
          (df[lon_col] < lonE) & (df[lon_col] > lonW)

### Realiza uma filtragem espacial e seleciona apenas as colunas de tempo, latitude e longitude:

In [8]:
focos = df[filter_][[time_col, lat_col, lon_col]].values

In [9]:
focos.shape

(3751, 3)

In [10]:
units = 'days since 1970-01-01'

In [11]:
def transform_time(time, units):
    times = pd.Series(time).dt.floor('d').astype(pd.datetime)
    times = nc.date2num(times, units, calendar='standard')
    return times

focos.T[0] = np.apply_along_axis(transform_time, 0, focos.T[0], units)

TypeError: dtype '<class 'datetime.datetime'>' not understood

In [ ]:
outputfile = 'arquivo.nc'

In [ ]:
if os.path.isfile(outputfile):
    os.remove(outputfile)

In [ ]:
ds = nc.Dataset(outputfile, mode='w')

### Calcula número de pontos de latitude e longitude:

In [ ]:
npointslat = np.int64(np.absolute(latS - latN) / dgrad) + 1
npointslon = np.int64(np.absolute(lonE - lonW) / dgrad) + 1

In [ ]:
lats = np.linspace(latS, latN, npointslat, dtype=np.float32)
lons = np.linspace(lonW, lonE, npointslon, dtype=np.float32)

In [ ]:
times = np.unique(focos.T[0])

In [ ]:
x_dim = ds.createDimension('lon', size=npointslon)
y_dim = ds.createDimension('lat', size=npointslat)
time_dim = ds.createDimension('time', size=None)

In [ ]:
lon_var = ds.createVariable('lon', np.double, dimensions=('lon',))
lon_var[:] = lons

lon_var.setncatts({
    'long_name': 'longitude',
    'units': 'degrees_east',
})

In [ ]:
lat_var = ds.createVariable('lat', np.double, dimensions=('lat',))
lat_var[:] = lats

lat_var.setncatts({
    'long_name': 'latitude',
    'units': 'degrees_north',
})

In [ ]:
times

In [ ]:
time_var = ds.createVariable('time', np.double, dimensions=('time',))
time_var[:] = times

time_var.setncatts({
    'units': units
})

In [ ]:
riscofogo = ds.createVariable('riscofogo', np.int32, dimensions=('time', 'lat', 'lon',), fill_value=-1)
riscofogo[:, :, :] = np.zeros(riscofogo.shape, dtype=np.int32)

riscofogo.setncatts({
    'long_name': 'Risco de fogo na vegetação - Resolução: {}km.'.format(int(dgrad * 100)),
    'fonte': 'Portal de dados abertos do Programa Queimadas/INPE.'
})

In [ ]:
def get_idx(dd, dd_array):
    idx = (np.absolute(dd_array - dd)).argmin()
    return idx


## Visualização de dados

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
xds = xr.open_dataset(outputfile)

### Gera uma visualização bidimensional dos dados que foram armazenados:

In [ ]:
xds.riscofogo.plot(x='lon', y='lat');